In [1]:
 # IMPORT PACKAGES
#!pip install selenium
#!pip install pyautogui
#!pip install webdriver-manager
from pathlib import Path
import os
import pandas as pd
import re
export = Path("csv")
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time 
import codecs
import urllib
from lxml import etree
import time
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())



[WDM] - Downloading:  96%|██████████████████████████████████████████████████████▋  | 6.20M/6.46M [00:10<00:00, 528kB/s]<ipython-input-1-4615d56b4b46>:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.46M/6.46M [00:29<00:00, 528kB/s]

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [3]:
#FUNCTION: DOWNLOAD CONTENT IN SEARCH RESULT PAGE 1
def down_1():
    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text

    # after loading is finished, download content in page 1
    final = os.path.abspath(export)  # SET THE SAVING LOCATION
    for i in range(1, 101):
        # SET THE LOCATION OF THE HEADLINE
        xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(i) + ']/td[3]/a'

        #Collapse Preventing
        load_time = 0  
        while load_time <= 5:  
            #
            driver.find_element(By.XPATH, xpath).click() 
            time.sleep(3)  


            check = driver.find_elements(By.XPATH,
                '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
            if len(check)==0:
                check = driver.find_elements(By.CLASS_NAME, 'article enArticle')
                
            wait_time = 0  
            while len(check) == 0 and wait_time <= 10:
                print(str(i) + 'Failed to load the article, retrying...')
                time.sleep(2)
                check = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                if len(check)==0:
                    check = driver.find_elements(By.CLASS_NAME, 'article enArticle')
                wait_time = wait_time + 1  
           
            if wait_time <= 10:
                print(str(i) + 'Parse successfully')
                content = check[0].text

                #Cleaning
                if content == '+ Related Dow Jones Intelligent Identifiers':
                    content = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[3]')[0].text
                else:
                    pass
                load_time = 6  
            else:
                driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
                time.sleep(1)
                load_time = load_time + 1
                print(str(i) + 'Retry to load the article')

        
        try:
            driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
        except:
            print('**Pass, Pass, Pass**') 

       

        final = final + '\n\n\n\n' + content + '\n\n\n\n'
        print(str(i) + ' is ok')

    final = final.lstrip('\n\n')
    #location = export + '_' +keyword+'_' +period + '_1.txt'
    
    location = os.path.join(export,  keyword + '-' + period + '-1.txt')
   
    file = open(location, mode='w', encoding='utf-8')
    file.write(final)


    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text

    print('**Page 1 is done**')

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.46M/6.46M [00:33<00:00, 202kB/s]


In [4]:
# SECOND PAGE AND LAST BUT ONE PAGE
def down_2():
    
    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
    driver.find_element(By.XPATH, '//*[@id="headlineHeader33"]/table/tbody/tr/td/a').click()  # 点击进入

    times = '1'  # one times = 10K articles
    page_i = 1
    # --------------------------------------------------------------------------------------------------------------

    keep_going = True
    while keep_going:
        page_i = page_i + 1 
        
        time.sleep(2)
        refresh_time = 0
        while refresh_time <=3:        
            content_judge1_1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
            content_judge2_2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
            refresh_time2 = 0
            while (content_judge1 == content_judge1_1) and (content_judge2 == content_judge2_2) and (refresh_time <= 24):
                print('page_' + str(page_i) + 'Loading')
                time.sleep(5)
                content_judge1_1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
                content_judge2_2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
                refresh_time = refresh_time + 1  

            if refresh_time <=24:
                print('page_' + str(page_i) + 'Successfully Loaded')
                refresh_time = 10
            else:
                refresh_time = refresh_time + 1
                driver.refresh()   
                time.sleep(10)

        results_bar = driver.find_element(By.CLASS_NAME, 'resultsBar').text
        results_bar = results_bar.split(" ")
        print(results_bar)
        num_results_on_page = 0
        if len(results_bar) == 6:
            num_results_on_page = int(results_bar[3])
            num_results_on_page = num_results_on_page%100
        if num_results_on_page==0:
            num_results = 100
        
        final =os.path.abspath(export)  # SET THE SAVING LOCATION
        for i in range(1, num_results_on_page+1):
            # Locate the Title
            xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(i) + ']/td[3]/a'

            load_time = 0  #
            while load_time <= 5: 
                
                driver.find_element(By.XPATH, xpath).click()  
                time.sleep(3)  

                check = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                if len(check)==0:
                    check = driver.find_elements(By.CLASS_NAME, 'article enArticle')
                wait_time = 0  
                while len(check) == 0 and wait_time <= 10:
                    print(str(i) + 'Loading failed, retrying')
                    time.sleep(2)
                    check = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                    if len(check)==0:
                        check = driver.find_elements(By.CLASS_NAME, 'article enArticle')
                    wait_time = wait_time + 1  
                #
                if wait_time <= 10:
                    print(str(i) + 'parsed successfully')
                    content = check[0].text

                    if content == '+ Related Dow Jones Intelligent Identifiers':
                        content = driver.find_elements(By.XPATH,
                            '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[3]')[0].text
                    else:
                        pass
                    load_time = 6  
                else:
                    driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
                    time.sleep(1)
                    load_time = load_time + 1
                    print(str(i) + 'Retrying')

            try:
                driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
            except:
                print('**Pass, Pass, Pass**')  



            final = final + '\n\n\n\n' + content + '\n\n\n\n'
            print(str(i) + ' is ok')

        final = final.lstrip('\n\n')
        #location = export+ '_' +keyword+'_' +period + '_' + str(page_i) + '.txt'
        location = os.path.join(export,  keyword + '-' +period + '-' + str(page_i) + '.txt')

        file = open(location, mode='w', encoding='utf-8')
        file.write(final)


        content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
        content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
        #driver.find_element(By.XPATH, '//*[@id="headlineHeader33"]/table/tbody/tr/td/a[2]').click()  
        
        #check if next item button exists. If so, click it
        test = driver.find_elements(By.CLASS_NAME, 'nextItem')
        if len(test)==0:
            keep_going=False
        else:
            driver.find_element(By.CLASS_NAME, 'nextItem').click()  

    print('**This time is well done**')
    # --------------------------------------------------------------------------------------------------------------


In [5]:
# LAST PAGE, WHICH HAS LESS THAN 100 NEWS
def down_3():
    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text

        # after loading is finished, download content in last page
    final = os.path.abspath(export) # SET THE SAVING LOCATION
    for i in range(1, n_news+1):
        # SET THE LOCATION OF THE HEADLINE
        xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(i) + ']/td[3]/a'

 
        load_time = 0  
        while load_time <= 5:  
            
            driver.find_element(By.XPATH, xpath).click() 
            time.sleep(3)  


            check = driver.find_elements(By.XPATH,
                '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
            wait_time = 0  
            while len(check) == 0 and wait_time <= 10:
                print(str(i) + 'Loading failed, retrying')
                time.sleep(2)
                check = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                wait_time = wait_time + 1 
          
            if wait_time <= 10:
                print(str(i) + 'parsed successfully')
                content = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')[0].text


                if content == '+ Related Dow Jones Intelligent Identifiers':
                    content = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[3]')[0].text
                else:
                    pass
                load_time = 6 
            else:
                driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click() 
                time.sleep(1)
                load_time = load_time + 1
                print(str(i) + 'Retrying')

        # 返回到总页面
        try:
            driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
        except:
            print('**Pass, Pass, Pass**') 


                    
        final = final + '\n\n\n\n' + content + '\n\n\n\n'
        print(str(i) + ' is ok')

        final = final.lstrip('\n\n')
        #location = export + '_' +keyword+'_' +period +'_last.txt'
        location = os.path.join(export,  keyword + '-' + period + '-1.txt')

        file = open(location, mode='w', encoding='utf-8')
        file.write(final)
    print('**LAST PAGE IS FINISHED**')    

In [93]:
#get spreadsheet of companies
workingdir = "C:/Users/arjain/Downloads/"
filename = 'Tech compnaies IPO_2010_2019.csv'
ipos = pd.read_csv(workingdir+filename)
ipos['Company Initial Public Offering Date'] = pd.to_datetime(ipos['Company Initial Public Offering Date'])    
ipos = ipos.drop_duplicates(subset=['Company Initial Public Offering Date', 'Global Company Key'], keep='first')
ipos = ipos.sort_values(by=['Company Initial Public Offering Date'], ascending=False)

In [94]:
# change the link to the final webpage for log-in
driver.get("http://libproxy.wustl.edu/login?url=https://global-factiva-com.libproxy.wustl.edu/en/sess/login.asp?xsid=S003sFp0HVyMTZyMTAoNp2nMTAtNHmm5DFHY96oYqZlNFFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFB")
driver.maximize_window()
window_handles = driver.window_handles
driver.switch_to.window(window_handles[0])

In [ ]:
#loop through companies and search on factiva
end = len(ipos)
ipos_i = 0

while ipos_i < end:

    #retrieve company info
    temp = ipos.iloc[ ipos_i ]
    comp_name = temp['Company Name']
    comp_name = comp_name.lower()
    ipo_date = pd.to_datetime(temp['Company Initial Public Offering Date'])    
    ipos_i += 1

    start_date = ipo_date - pd.Timedelta(30, "d")
    end_date = ipo_date + pd.Timedelta(30, "d")

    start_date = start_date.strftime("%Y%m%d")
    end_date = end_date.strftime("%Y%m%d")
    
    #for file naming
    keyword = comp_name
    period = '{start} to {end}'.format(start=start_date, end=end_date)
    
    time.sleep(2)
    
    #input query into search form
    query = ''''{comp_name}' and date from {start} to {end}'''.format(comp_name = comp_name, start=start_date, end=end_date)
    freetext=driver.find_element(By.CLASS_NAME, 'ace_text-input')
    freetext.send_keys(query)

    time.sleep(0.2)
    driver.find_element(By.XPATH, "//select[@name='dr']/option[text()='All Dates']").click()
    time.sleep(0.2)
    driver.find_element(By.XPATH, "//select[@name='isrd']/option[text()='Identical']").click()

    # click search button
    time.sleep(2)
    elem = driver.find_element(By.XPATH, '//li[@id="btnSBSearch"]')

    actions = ActionChains(driver)
    actions.click(elem).perform()


    # display option
    time.sleep(2)
    elem = driver.find_element(By.XPATH, '//span[@id="articleViewAs"]')

    actions = ActionChains(driver)
    actions.click(elem).perform()

    time.sleep(1)
    driver.find_element(By.XPATH, "//a[@onclick=\"changeFormat('FULR', true, this);return false;\"]").click()
    
    #find how many results are on the page
    results_bar = driver.find_element(By.CLASS_NAME, 'resultsBar').text
    results_bar = results_bar.split(" ")
    print(results_bar)
    num_results_on_page = 0
    if len(results_bar) == 6:
        num_results_on_page = int(results_bar[3])
        
    #parse all results for query
    i = 1
    if num_results_on_page < 100 and num_results_on_page != 0:
        n_news=num_results_on_page
        #set n_news to the number of news in the last page shown
        down_3()
    elif num_results_on_page >= 100:
        down_1()
        down_2()
    
    # clear previous search info
    time.sleep(2)
    driver.find_element(By.XPATH, '//a[@title="Search"]').click()
    time.sleep(2)
    freetext=driver.find_element(By.CLASS_NAME, 'ace_text-input')
    freetext.send_keys(Keys.CONTROL, 'a')
    freetext.send_keys(Keys.BACKSPACE)


['Headlines', '1', '-', '15', 'of', '19']
1parsed successfully
1 is ok
2parsed successfully
2 is ok
3parsed successfully
3 is ok
4parsed successfully
4 is ok
5parsed successfully
5 is ok
6parsed successfully
6 is ok
7parsed successfully
7 is ok
8parsed successfully
8 is ok
9parsed successfully
9 is ok
10parsed successfully
10 is ok
11parsed successfully
11 is ok
12parsed successfully
12 is ok
13parsed successfully
13 is ok


In [91]:
#if there is some issue look at what company and time period it stops at 
print(query)

'sprout social inc' and date from 20191113 to 20200112


In [32]:
#Only when the download stops unexpectedly, run this line
def down_exp():
    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text

    page_i=page_break
    
    # after loading is finished, download content in page 1
    final = os.path.abspath(export)  # SET THE SAVING LOCATION
    for i in range(1, 101):
        # SET THE LOCATION OF THE HEADLINE
        xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(i) + ']/td[3]/a'

        load_time = 0  
        while load_time <= 5: 
            driver.find_element(By.XPATH, xpath).click()  
            time.sleep(3)  

            check = driver.find_elements(By.XPATH,
                '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
            wait_time = 0  
            while len(check) == 0 and wait_time <= 10:
                print(str(i) + 'Loading failed, retrying')
                time.sleep(2)
                check = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                wait_time = wait_time + 1  
            if wait_time <= 10:
                print(str(i) + 'Successful')
                content = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')[0].text

                if content == '+ Related Dow Jones Intelligent Identifiers':
                    content = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[3]')[0].text
                else:
                    pass
                load_time = 6  
            else:
                driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click() 
                time.sleep(1)
                load_time = load_time + 1
                print(str(i) + 'Retrying')

        try:
            driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click()  
        except:
            print('**Pass, Pass, Pass**')  

      
        final = str(final) + '\n\n\n\n' + content + '\n\n\n\n'
        print(str(i) + ' is ok')

    final = final.lstrip('\n\n')
    #location = export+'_' +keyword +'_'+ period + '_'+str(page_break)+'.txt'
    location = os.path.join(export,  '_' +keyword+'_' +period + '_' + str(page_i) + '.txt')

    file = open(location, mode='w', encoding='utf-8')
    file.write(final)

    content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
    content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
    
    
    # --------------------------------------------------------------------------------------------------------------
    
    
    driver.find_element_by_class_name('nextItem').click()
    
    
    while page_i <=99:
        page_i = page_i + 1 
        time.sleep(2)
        refresh_time = 0
        while refresh_time <=3:         
            content_judge1_1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
            content_judge2_2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
            refresh_time2 = 0
            while (content_judge1 == content_judge1_1) and (content_judge2 == content_judge2_2) and (refresh_time <= 24):
                print('page_' + str(page_i) + 'Loading')
                time.sleep(5)
                content_judge1_1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
                content_judge2_2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
                refresh_time = refresh_time + 1 
            if refresh_time <=24:
                print('page_' + str(page_i) + 'Successfully Loaded')
                refresh_time = 10
            else:
                refresh_time = refresh_time + 1
                driver.refresh()    
                time.sleep(10)

        
        final = export  # SET THE SAVING LOCATION
        for i in range(1, 101):
            
            xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(i) + ']/td[3]/a'

            
            load_time = 0  
            while load_time <= 5:  
                
                driver.find_element(By.XPATH, xpath).click() 
                time.sleep(3)  

                
                check = driver.find_elements(By.XPATH,
                    '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                wait_time = 0  
                while len(check) == 0 and wait_time <= 10:
                    print(str(i) + 'Retrying')
                    time.sleep(2)
                    check = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')
                    wait_time = wait_time + 1  
                
                if wait_time <= 10:
                    print(str(i) + 'Parsed successfully')
                    content = driver.find_elements(By.XPATH,
                        '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[2]')[0].text

                    
                    if content == '+ Related Dow Jones Intelligent Identifiers':
                        content = driver.find_elements(By.XPATH,
                            '/html/body/form[2]/div[2]/div[2]/div[5]/div[2]/div[3]/div/div[2]/div/div[2]/div[3]')[0].text
                    else:
                        pass
                    load_time = 6 
                else:
                    driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click() 
                    time.sleep(1)
                    load_time = load_time + 1
                    print(str(i) + 'Retrying')

            
            try:
                driver.find_element(By.XPATH, '//*[@id="returnToHeadlines"]/a').click() 
            except:
                print('**Pass, Pass, Pass**') 

            
            
            final = str(final) + '\n\n\n\n' + content + '\n\n\n\n'
            print(str(i) + ' is ok')

        final = final.lstrip('\n\n')
        location = os.path.join(export,  '_' +keyword+'_' +period + '_' + str(page_i) + '.txt')

        file = open(location, mode='w', encoding='utf-8')
        file.write(final)


        
        content_judge1 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
        content_judge2 = driver.find_element(By.XPATH, '//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
        driver.find_element(By.XPATH, '//*[@id="headlineHeader33"]/table/tbody/tr/td/a[2]').click()

    print('**This time is well done**')


In [ ]:
#when it stops unexpectedly
page_break=18
#input the page number of the code stops
down_exp()

C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge1 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge2 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(xpath).click()
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  check = driver.find_elements_by_xpath(
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/17

1Successful
1 is ok
2Successful
2 is ok
3Successful
3 is ok
4Successful
4 is ok
5Successful
5 is ok
6Successful
6 is ok
7Successful
7 is ok
8Successful
8 is ok
9Successful
9 is ok
10Successful
10 is ok
11Successful
11 is ok
12Successful
12 is ok
13Successful
13 is ok
14Successful
14 is ok
15Successful
15 is ok
16Successful
16 is ok
17Successful
17 is ok
18Successful
18 is ok
19Successful
19 is ok
20Successful
20 is ok
21Successful
21 is ok
22Successful
22 is ok
23Successful
23 is ok
24Successful
24 is ok
25Successful
25 is ok
26Successful
26 is ok
27Successful
27 is ok
28Successful
28 is ok
29Successful
29 is ok
30Successful
30 is ok
31Successful
31 is ok
32Successful
32 is ok
33Successful
33 is ok
34Successful
34 is ok
35Successful
35 is ok
36Successful
36 is ok
37Successful
37 is ok
38Successful
38 is ok
39Successful
39 is ok
40Successful
40 is ok
41Successful
41 is ok
42Successful
42 is ok
43Successful
43 is ok
44Successful
44 is ok
45Successful
45 is ok
46Successful
46 is ok
47Succ

C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:61: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge1 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge2 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:68: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('nextItem').click()
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge1_1 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbod

page_19Successfully Loaded


C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:107: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  check = driver.find_elements_by_xpath(
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:119: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content = driver.find_elements_by_xpath(
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:124: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content = driver.find_elements_by_xpath(
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:137: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="returnToHeadlines"]/a').click()


...

C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:155: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge1 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:156: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge2 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:157: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="headlineHeader33"]/table/tbody/tr/td/a[2]').click()  # 进入下一页继续爬取


page_20Successfully Loaded
1Parsed successfully
1 is ok
2Parsed successfully
2 is ok
3Parsed successfully
3 is ok
4Parsed successfully
4 is ok
5Parsed successfully
5 is ok
6Parsed successfully
6 is ok
7Parsed successfully
7 is ok
8Parsed successfully
8 is ok
9Parsed successfully
9 is ok
10Parsed successfully
10 is ok
11Parsed successfully
11 is ok
12Parsed successfully
12 is ok
13Parsed successfully
13 is ok
14Parsed successfully
14 is ok
15Parsed successfully
15 is ok
16Parsed successfully
16 is ok
17Parsed successfully
17 is ok
18Parsed successfully
18 is ok
19Parsed successfully
19 is ok
20Parsed successfully
20 is ok
21Parsed successfully
21 is ok
22Parsed successfully
22 is ok
23Parsed successfully
23 is ok
24Parsed successfully
24 is ok
25Parsed successfully
25 is ok
26Parsed successfully
26 is ok
27Parsed successfully
27 is ok
28Parsed successfully
28 is ok
29Parsed successfully
29 is ok
30Parsed successfully
30 is ok
31Parsed successfully
31 is ok
32Parsed successfully
32 is ok

C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:82: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge1_1 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[1]/td[3]/a').text
C:\Users\kyle0\AppData\Local\Temp/ipykernel_7364/1743551726.py:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_judge2_2 = driver.find_element_by_xpath('//*[@id="headlines"]/table/tbody/tr[2]/td[3]/a').text


page_21Successfully Loaded
1Parsed successfully
1 is ok
2Parsed successfully
2 is ok
3Parsed successfully
3 is ok
4Parsed successfully
4 is ok
5Parsed successfully
5 is ok
6Parsed successfully
6 is ok
7Parsed successfully
7 is ok
8Parsed successfully
8 is ok
9Parsed successfully
9 is ok
10Parsed successfully
10 is ok
11Parsed successfully
11 is ok
12Parsed successfully
12 is ok
13Parsed successfully
13 is ok
14Parsed successfully
14 is ok
15Parsed successfully
15 is ok
16Parsed successfully
16 is ok
17Parsed successfully
17 is ok
18Parsed successfully
18 is ok
19Parsed successfully
19 is ok
20Parsed successfully
20 is ok
21Parsed successfully
21 is ok
22Parsed successfully
22 is ok
23Parsed successfully
23 is ok
24Parsed successfully
24 is ok
25Parsed successfully
25 is ok
26Parsed successfully
26 is ok
27Parsed successfully
27 is ok
28Parsed successfully
28 is ok
29Parsed successfully
29 is ok
30Parsed successfully
30 is ok
31Parsed successfully
31 is ok
32Parsed successfully
32 is ok

65Parsed successfully
65 is ok
66Parsed successfully
66 is ok
67Parsed successfully
67 is ok
68Parsed successfully
68 is ok
69Parsed successfully
69 is ok
70Parsed successfully
70 is ok
71Parsed successfully
71 is ok
72Parsed successfully
72 is ok
73Parsed successfully
73 is ok
74Parsed successfully
74 is ok
75Parsed successfully
75 is ok
76Parsed successfully
76 is ok
77Parsed successfully
77 is ok
78Parsed successfully
78 is ok
79Parsed successfully
79 is ok
80Parsed successfully
80 is ok
81Parsed successfully
81 is ok
82Parsed successfully
82 is ok
83Parsed successfully
83 is ok
84Parsed successfully
84 is ok
85Parsed successfully
85 is ok
86Parsed successfully
86 is ok
87Parsed successfully
87 is ok
88Parsed successfully
88 is ok
89Parsed successfully
89 is ok
90Parsed successfully
90 is ok
91Parsed successfully
91 is ok
92Parsed successfully
92 is ok
93Parsed successfully
93 is ok
94Parsed successfully
94 is ok
95Parsed successfully
95 is ok
96Parsed successfully
96 is ok
97Parsed